In [ ]:
!pip install sentence-transformers


In [4]:
import re
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/train.csv")

In [ ]:
df.head()

In [ ]:
df['text'] = df['text'].apply(lambda x: re.sub(r'[\n\\/]+', '', str(x)))

In [ ]:
df = df.drop_duplicates()


In [ ]:
df = df.iloc[:10000]

In [ ]:
# Filter the DataFrame to create a DataFrame for positive reviews
df_positive = df[df['label'] == 1]

# Filter the DataFrame to create a DataFrame for negative reviews
df_negative = df[df['label'] == 0]

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Generate Embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
positive_embeddings = model.encode(df_positive['text'].tolist())
negative_embeddings = model.encode(df_negative['text'].tolist())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Initialize to store matched pairs
matched_pairs = []

# Similarity threshold
gamma = 0.8
for i, x_embedding in enumerate(positive_embeddings):
    similarities = cosine_similarity([x_embedding], negative_embeddings)[0]
    max_index = np.argmax(similarities)
    max_similarity = similarities[max_index]
    if max_similarity > gamma:
        # Append only the sentences to matched_pairs, not the similarity score
        matched_pairs.append((df_positive.iloc[i]['text'], df_negative.iloc[max_index]['text']))

# Create a DataFrame from the matched pairs
df_matched = pd.DataFrame(matched_pairs, columns=['Sentence_from_X', 'Matched_Sentence_from_Y'])

# Display the DataFrame
print(df_matched)

In [ ]:
df_matched.to_excel('data.xlsx', index=False)